In [1]:
import numpy as np
from astropy.table import Table
from astropy.constants import G, c, m_p, sigma_T, M_sun, L_sun
import astropy.units as u

In [2]:
tab = Table.read("SMBH_masses_and_Lbol_from_Guodong.txt", format='ascii')

In [3]:
L_edd_fact = 4*np.pi*G*M_sun*m_p*c/sigma_T/L_sun
log_Ledd_fact = np.log10(L_edd_fact)
print((L_edd_fact).to(u.dimensionless_unscaled))

32838.69330843236


In [4]:
#Get the Eddington luminosities in L_sun units.
tab['log_Ledd/Lsun'] = tab['log_MBH/M_sun'] + log_Ledd_fact

#Now, get the Eddington ratio. 
tab['ledd_rat'] = 10**(tab['log_Lbol/L_sun'] - tab['log_Ledd/Lsun'])
log_ledd_rat = tab['log_Lbol/L_sun'] - tab['log_Ledd/Lsun']

#The Eddington ratio error in log should just be the addition of the 0.4 and 0.2 dex of the MBH and Lbol. 
log_ledd_rat_err = (0.4**2+0.2**2)**0.5
print(log_ledd_rat_err)
tab['ledd_rat_err_low'] = tab['ledd_rat'] - 10**(log_ledd_rat-log_ledd_rat_err)
tab['ledd_rat_err_hig'] = 10**(log_ledd_rat+log_ledd_rat_err) - tab['ledd_rat']


0.447213595499958


In [5]:
tab['log_Ledd/Lsun'].format = '%.1f'
tab['ledd_rat'].format = '%.3f'
tab['ledd_rat_err_low'].format = '%.3f'
tab['ledd_rat_err_hig'].format = '%.3f'

In [6]:
tab.show_in_notebook()

idx,ID,log_MBH/M_sun,log_MBH_err,log_Lbol/L_sun,log_Lbol_err,log_Ledd/Lsun,ledd_rat,ledd_rat_err_low,ledd_rat_err_hig
,,,,,,,,,
0,W0019-1046,9.8,0.4,13.3,0.2,14.3,0.096,0.062,0.173
1,W0116-0505,9.4,0.4,14.1,0.2,13.9,1.526,0.981,2.748
2,W0204-0506,8.8,0.4,13.7,0.2,13.3,2.419,1.555,4.355
3,W0220+0137,9.3,0.4,13.9,0.2,13.8,1.212,0.779,2.183
4,W0831+0140,9.4,0.4,14.4,0.2,13.9,3.045,1.958,5.482


In [7]:
tab['log_Lbol/L_sun'].data

array([13.3, 14.1, 13.7, 13.9, 14.4])

## W0019-1046

In [8]:
#Let's figure out what CIV line-width we would need to make all the numbers for W0019-1046 match. 
alpha = 4.7
Lbol = 10**tab['log_Lbol/L_sun'] * L_sun
L5100 = alpha * Lbol

In [9]:
#Conversion taken from my SED template. 
L1450 = (2.366959e-01/1.572620e-01) * L5100 * 1450/5100.
print(L1450)

[1.53615328e+40 9.69247197e+40 3.85864259e+40 6.11553638e+40
 1.93390241e+41] W


In [13]:
log_FWHM_CIV = (tab['log_MBH/M_sun'] - 6.353 - 0.599 * np.log10(L1450/(1e44 * u.erg/u.s)))/2
FWHM_CIV = 1e3 * u.km/u.s * 10**(log_FWHM_CIV)
print(FWHM_CIV)
print(FWHM_CIV[0]/FWHM_CIV)

[5877.09689755 2135.8111584  1410.46674501 2185.05740524 1736.65220116] km / s
[1.         2.75169313 4.16677452 2.6896762  3.38415308]


In [11]:
#The actual measurement is 6844 +1402 -3028. 
2 * np.log10((6844.-3000.)/6844.), 2 * np.log10((6844.+1402)/6844.)

(-0.5010532437451097, 0.16186665919255397)